In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle
import torch

import torch
import torch.nn as nn

# Models

Version 1:
* LGBM
* Full sample
* Added features derived from cross-sectional distributions within- and across-clusters
* Score: 0.098

Version 2:
* LGBM
* Full sample
* Added features derived from cross-sectional distribution of features (no clustering)
* Score: 0.098

Version 3: 
* LGBM
* Full sample
* No derived features (only the 300 original)
* Score: 0.098

Version 4: 
* LGBM
* Sample: `time_id` > 400
* No derived features (only the 300 original)
* Score: 0.092

Version 5: 
* Neural net
* Sample: full
* No derived features (only the 300 original)
* Score: 0.1075

Version 6: 
* Ensemble: 50% LGBM and 50% DNN
* Sample: full
* No derived features (only the 300 original)
* Score: 0.1108




# Submission

In [2]:
class Model(nn.Module):
    def __init__(self, num_features):
        super().__init__()

        dropouts = [0.10, 0.10, 0.10, 0.10]
        hidden_size = [512, 256, 128]
        
        layers = [nn.BatchNorm1d(num_features)]
        in_size = num_features

        for i in range(len(hidden_size)):
            out_size = hidden_size[i]
            layers.append(nn.Dropout(dropouts[i]))
            layers.append(nn.Linear(in_size, out_size))
            layers.append(nn.BatchNorm1d(out_size))
            layers.append(nn.SiLU()) 
            in_size = out_size

        layers.append(nn.Dropout(dropouts[-1]))
        layers.append(nn.Linear(in_size, 1))
        
        self.model = torch.nn.Sequential(*layers)

    def forward(self, cat, cont):
        x = self.model(cont)
        return x


In [3]:
nn_version = 1
nn_model = pickle.load(open(f'../input/nnversion{nn_version}/NNresults{nn_version}.pkl', 'rb'))

lgbm_version = 3
lgbm_model, importance = pickle.load(open(f'../input/lgbm-version{lgbm_version}/lgbm_results{lgbm_version}.pkl', 'rb'))

In [4]:
def predict_lgbm(df, columns, model):
    # For LGBM models, returns the prediction over columns in df
    
    return model.predict(df[columns])

In [5]:
def predict_nn(df, columns, mod):
    # Returns PyTorch predictions
    
    X_test = df.loc[:, columns].values
    
    pred_vector = (
        mod(torch.tensor([]), torch.tensor(X_test, dtype=torch.float))
             .detach()
             .cpu()
             .numpy()
        )
    
    return pred_vector[:, 0]

In [6]:
import ubiquant

env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

for (test_df, sample_prediction_df) in iter_test:
    
    
    
    #transform(test_df) # generate derived features
    
    features         = [col for col in test_df if col.startswith('f_')]
    #derived_features = [col for col in test_df if col.startswith('all')]
    #cluster_features = [col for col in test_df if col.startswith('clust')]
    
    
    test_df['target_lgbm'] = predict_lgbm(test_df, features, lgbm_model)
    test_df['target_nn']   = predict_nn(  test_df, features, nn_model)
    
    test_df['target'] = test_df[['target_lgbm', 'target_nn']].mean(axis=1)
    
    env.predict(test_df[['row_id','target']])

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
